# 

In [142]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs
import ElTiempoFunciones as etf

import os as os
import csv
import json

In [143]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    
#for i in empresas['LaRepublica']:
    

In [144]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [145]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.eltiempo.com/buscar?q={empresa}')
# time.sleep(2)
driver.implicitly_wait(10) #Nueva metodología de wait

In [146]:
# El número total de ventanas que tiene la búsqueda

total_num_ventanas = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div/div[11]/div/ul/li[10]/a').get_attribute('href')
print(total_num_ventanas)

https://www.eltiempo.com/buscar/2199?q=macbook


In [147]:
# Recolecta datos hasta la ventana 20 del Tiempo
total_num_ventanas = f'https://www.eltiempo.com/buscar/2?q={empresa}'

In [148]:
titulares=[]
get_url = driver.current_url
pag_actual = 1

while(get_url != total_num_ventanas):
        buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div')
        articulos = buscar.find_elements(By.CLASS_NAME,"listing")

        for articulos in articulos:
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)
                if not(bs.existedb(url, "eltiempo")):
                        titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                        # print(titulo)
                        resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                        # print(resumen)
                        fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                        # print(fechaPub)
                        tema = articulos.find_element(By.CLASS_NAME,"category").text
                        # print(tema)
                        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                        'Titulo':titulo,
                                        'Fecha Publicacion':fechaPub,
                                        'Tema':tema,
                                        'URL':url,
                                        'Resumen':resumen,
                                        'Empresa':empresa})

        pag_actual=pag_actual+1
        driver.get(f'https://www.eltiempo.com/buscar/{pag_actual}?q={empresa}')
        driver.implicitly_wait(10) # Wait mientras abre la ventana
        get_url = driver.current_url
# driver.quit()
# print(titulares)

In [149]:
buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div')
articulos = buscar.find_elements(By.CLASS_NAME,"listing")

for articulos in articulos:
        if not(bs.existedb(url, "eltiempo")):
                titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                # print(titulo)
                resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                # print(resumen)
                fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                # print(fechaPub)
                tema = articulos.find_element(By.CLASS_NAME,"category").text
                # print(tema)
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)

                titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                'Titulo':titulo,
                                'Fecha Publicacion':fechaPub,
                                'Tema':tema,
                                'URL':url,
                                'Resumen':resumen,
                                'Empresa':empresa})

driver.delete_all_cookies()


### Extrae info de cada URL

In [150]:
# Busca cada articulo y las almacena en la lista de titulares
for tit in titulares:
    # clear all cookies in scope of session
    driver.delete_all_cookies()
    driver.get(tit['URL'])
    driver.implicitly_wait(10) #Nueva metodología de wait

    # agregar contenido al dict de titulares
    # tit['Contenido'] = bs.obtener_contenido_eltiempo(driver)
    
    #agregar lista de URLs de noticias relacionadas
    # tit['RelNewsUrls'] = bs.obtener_articulos_relacionados_eltiempo(driver)


### Para agregar la info en un archivo CSV

In [151]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema']

# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

In [152]:
titulares[-1]

{'Fecha Extraccion': '2023-01-26 22:07:34',
 'Titulo': 'Odebrecht: se cayó el principio de oportunidad entre Fiscalía y David Portilla',
 'Fecha Publicacion': 'ENERO 25 DE 2023',
 'Tema': 'UNIDAD INVESTIGATIVA',
 'URL': 'https://www.eltiempo.com/unidad-investigativa/odebrecht-se-cae-principio-de-oportunidad-con-david-portilla-736704',
 'Resumen': 'Pide que le devuelvan $ 100 millones. Fiscal del caso no ha entregado declaración de brasileño.',
 'Empresa': 'macbook'}